Logging
=======

![](https://d33wubrfki0l68.cloudfront.net/59b1aae2adcfe4f06270b99a2789012ed64bec1f/4d0ad/images/docs/user-guide/logging/logging-node-level.png)

Quelle: https://kubernetes.io/docs/concepts/cluster-administration/logging/
- - -

Mithilfe von Anwendungs- und Systemprotokollen können Sie besser verstehen, was in Ihrem Cluster geschieht.

Die Protokolle sind besonders nützlich zum Debuggen von Problemen und zum Überwachen der Clusteraktivität. 

Die einfachste und am weitesten verbreitete Protokollierungsmethode für containerisierte Anwendungen besteht darin, in die Standardausgabe (stdout) und die Standardfehlerströme (stderr) zu schreiben.

Um dies zu Demonstrieren starten wir einen Container mit einer einfachen Endlosschleife, welche einen Zähler und das Datum ausgibt.

In [ ]:
! kubectl apply -f https://raw.githubusercontent.com/mc-b/duk/master/logging/counter-pod.yaml

und geben sein Log aus:

In [ ]:
! kubectl logs counter

Nach dem beenden des Container bzw. Pods stehen auch die Logs nicht mehr zur Verfügung.

Die zweite Anweisung sollte also entweder einen Fehler oder keine Ausgabe mehr bringen:

In [ ]:
! kubectl delete -f https://raw.githubusercontent.com/mc-b/duk/master/logging/counter-pod.yaml
! kubectl logs counter

***
Fluent Bit
----------

![](https://fluentbit.io/assets/img-flb/flb_002.png)

Quelle: https://fluentbit.io/

- - -

Fluent Bit ist ein Open Source- und plattformübergreifender Protokollprozessor und -weiterleiter , mit dem Sie Daten / Protokolle aus verschiedenen Quellen sammeln, vereinheitlichen und an mehrere Ziele senden können.

Fluent Bit soll Sicherstellen, dass wir die Logs auch nach Beendigung von Container/Pods zur Verfügung haben.

Fluent Bit ist in der [duk-demo](https://github.com/mc-b/duk-demo) Umgebung bereits installiert. Siehe Projekt [duk](https://github.com/mc-b/duk/tree/master/logging).


*** 
Logs
----

Zu Testzwecken ist Fluent Bit so konfiguriert, dass es die Logs sammelt und diese auf `stdout` des Fluent Bit Pods ausgibt.

    output-stdout.conf: |
      [OUTPUT]
        Name            stdout
        Match           *
        Logstash_Format On
        Replace_Dots    On
        Retry_Limit     False

So müssen wir nur noch das Log des Fluent Bit Pods anschauen um alle Logsausgaben anzusehen:

In [ ]:
! kubectl logs --tail=20 -l k8s-app=fluent-bit-logging

*** 
ElasticSearch und Kibana
------------------------

Um die Logs zu Speichern und zu Visualisieren, braucht es weitere Produkte:
* ElasticSearch - eine verteilte RESTful-Suchmaschine und -Analytics Engine
* Kibana - die visuelle Analyse der Elasticsearch-Daten in Echtzeit.

Nur Kibana hat ein UI welcher wir zuerst freischalten müssen:

In [ ]:
! kubectl get service quickstart-kb-http -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -

In [ ]:
! echo "Kibana User/PW: " elastic/$(kubectl get secret quickstart-es-elastic-user -o=jsonpath='{.data.elastic}' | base64 --decode; echo)
! echo "Kibana UI     : " $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e "s/6443/$(kubectl get service quickstart-kb-http -o=jsonpath='{ .spec.ports[0].nodePort }')/g")